# google drive

utilities for google drive




## basic needs

authorization, listing, writing, copying, etc


### authorization
authorize google drive?


#### the code 



In [1]:
var fs = require('fs');
var path = require('path');
var {google} = require('googleapis');
var {GoogleAuth} = require('google-auth-library');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var credentials;
if(fs.existsSync('./sheet to web-8ca5784e0b05.json')) {
    credentials = path.resolve('./sheet to web-8ca5784e0b05.json');
} else {
    credentials = path.join(PROFILE_PATH, '.credentials/sheet to web-8ca5784e0b05.json');
}

var GOOGLE_AUTH_SCOPE = [
    'https://www.googleapis.com/auth/drive'
];

function authorizeDrive() {
    return new GoogleAuth({
        keyFile: credentials,
        scopes: GOOGLE_AUTH_SCOPE
    }).getClient()
        .then(client => google.drive({version: 'v3', auth: client}))
}

module.exports = authorizeDrive;


[Function: authorizeDrive]

#### test list google drive?


In [ ]:
var importer = require('../Core');
var listDrive = importer.import('list google drive files');
var insertPermission = importer.import('insert google drive permissions');

describe('list google drive files', () => {
    
    it('should list files', () => {
        return listDrive()
            .then(r => importer
                  .runAllPromises(r.filter(r => r.name.includes('Untitled'))
                                           .map(f => resolve => insertPermission(f.id, 'megamindbrian@gmail.com')
                                                .then(resolve))))
    })
})


### list google drive files?


In [ ]:
var util = require('util');
var importer = require('../Core');
var authorizeDrive = importer.import('authorize google drive');

function listDrive() {
    return authorizeDrive()
        .then(drive => util.promisify(drive.files.list.bind(drive))({}))
        .then(r => r.data.files || [])
}

module.exports = listDrive;


### insert google drive permissions?


In [ ]:
var util = require('util');
var importer = require('../Core');
var authorizeDrive = importer.import('authorize google drive');

function insertPermission(fileId, email) {
    return authorizeDrive()
        .then(drive => util.promisify(drive.permissions.create.bind(drive))({
            resource: {
                'type': 'user',
                'role': 'owner',
                'emailAddress': email
            },
            fileId: fileId,
            fields: 'id',
            transferOwnership: true
        }))
        .then(r => r.data.id)
}

module.exports = insertPermission;


### create a sheet in google drive?


In [ ]:
var util = require('util');
var importer = require('../Core');
var authorizeSheets = importer.import('authorize sheets api');
var insertPermission = importer.import('insert google drive permissions');

function createSheet(email) {
    var sheets;
    
    return authorizeSheets()
        .then(s => sheets = s)
        .then(() => util.promisify(sheets.spreadsheets.create.bind(sheets))())
        .then(r => insertPermission(r.data.spreadsheetId, 'megamindbrian@gmail.com'))
        .then(r => insertPermission(r.data.spreadsheetId, email))
}

module.exports = createSheet;


#### test google sheet create?


In [ ]:
var importer = require('../Core');
var createSheet = importer.import('create a sheet in google drive');

describe('create a new marketing sheet', () => {
    
    it('should create a sheet', () => {
        return createSheet('bjcullinan@gmail.com');
    })
})


### copy a file on google drive?


In [ ]:
var util = require('util');
var importer = require('../Core');
var authorizeDrive = importer.import('authorize google drive');

function copyFile(fileId, title) {
    return authorizeDrive()
        .then(drive => util.promisify(drive.files.copy.bind(drive))({
            fileId: fileId,
            resource: {name: title},
            convert: true
        }))
        .then(r => r.data.id)
}

module.exports = copyFile;


## advanced tools

comparing, cateloging, sharing, converting, etc



### compare a folder with local

What is the purpose 🐬 of this? Well, Google Drive sync is awful for these reason:

1. Drive creates an MD5 hash for every single file. Anyone familiar with rsync principles knows there are plenty of faster metrics to compare before hashing a file, such as, does it exist? Did the file time change?
2. It stores the list of files in separate database, and is completely useless without that database, has to be rebuilt from scratch.
3. Can't upload files while it is searching for files, many FTP tools use to have this problem, and SourceTree still has this problem.
4. Slows to a crawl the more files you have (~300,000), rendering it useless for backup purposes.
5. Insists on recreating a cache every time you try to use it on a different computer because of that missing database.
6. It created 2 directories with the same name, I can't trust this service anymore.
7. Apparently, somebody decided that "On demand" isn't the same as "On command", can't sync the files I want when I need them, always automatic, no way to control queuing. Having a nice little settings dialog and task bar icon doesn't excuse this.
8. Of course! Good Google sync is only available for paying customers. Even though I am paying for extra storage every month?  How much sense does this make?

Let's separate the databasing features from the syncing features. And separate the uploading features from the databasing features. And, if we need any merging features, those should be separate too. Because of my experience with my mediaserver project, I have the necessary skills to implement this concisely.

* scan - look for files and folders
* upload - send to Google drive
* merge - do the actual syncing
* extract - take real documents out of drive, like a download but it has to be converted from Google's proprietary internal format.



#### the code 

compare google drive?

merge google drive?


In [ ]:
var fs = require('fs');
var path = require('path');
var {GoogleAuth} = require('google-auth-library');
var importer = require('../Core')
var getRpcFromSpec = importer.import('get rpc from spec')
var authorize = importer.import('google oauth token client')

// TODO: pattern recognized! create a "google" object that does this for every service in the Google discovery list
var google = {drive: ({version, auth}) => getRpcFromSpec(require(path.join(__dirname, `../Resources/APIs/drive.${version}.json`)), auth)}

//var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
/*
var credentials;
if(fs.existsSync('./sheet to web-8ca5784e0b05.json')) {
    credentials = path.resolve('./sheet to web-8ca5784e0b05.json');
} else {
    credentials = path.join(PROFILE_PATH, '.credentials/sheet to web-8ca5784e0b05.json');
}
*/

var GOOGLE_AUTH_SCOPE = [
    'https://www.googleapis.com/auth/drive'
];

async function listDriveFiles(drive, folder, pageToken) {
    var result = await drive.files.list({
        fields: 'files(kind,id,name,modifiedTime,md5Checksum,mimeType,owners,size,parents,originalFilename)',
        pageToken: pageToken,
        q: `'${folder}' in parents and trashed = false`
        //driveId: 'my-drive',
        //includeTeamDriveItems: true,
        //corpora: 'drive',
        //supportsAllDrives: true
    })
    if(result.data.nextPageToken) {
        var moreFiles = listDriveFiles(drive, folder, result.data.nextPageToken)
        return result.data.files.concat(moreFiles)
    } else {
        return result.data.files
    }
}

async function listDrive(folder = 'root') {
    var client = await authorize(GOOGLE_AUTH_SCOPE)
    var drive = await google.drive({version: 'v3', auth: client})
    var files = await listDriveFiles(drive, folder)
    return files
}

module.exports = listDrive

if(typeof $$ != 'undefined') {
    listDrive()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}




download all docs as actual data files?



In [ ]:
const listDrive = importer.import('merge google drive')
const getRpcFromSpec = importer.import('get rpc from spec')
const authorize = importer.import('google oauth token client')
const google = {drive: ({version, auth}) => getRpcFromSpec(require(path.join(__dirname, `../Resources/APIs/drive.${version}.json`)), auth)}
const { safeurl } = importer.import('domain cache tools')
const GOOGLE_AUTH_SCOPE = [
  'https://www.googleapis.com/auth/drive'
]

const PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
const DOWNLOAD_PATH = path.join(PROFILE_PATH, 'Downloads')

async function convertGoogleDoc(fileId, outType) {
  let client = await authorize(GOOGLE_AUTH_SCOPE)
  let drive = await google.drive({version: 'v3', auth: client})
  return await drive.files.export({
    fileId: fileId,
    mimeType: outType
  })
}


async function downloadDocs(folder = 'root') {

  let drive = await listDrive(folder)

  for(let i = 0; i < drive.length; i++) {
    if(drive[i].mimeType == 'application/vnd.google-apps.document') {
      console.log('Exporting ' + drive[i].name)
      try {
        let response = await convertGoogleDoc(drive[i].id, 'application/vnd.openxmlformats-officedocument.wordprocessingml.document')
        let data = await response.data.arrayBuffer()
        fs.writeFileSync(path.join(DOWNLOAD_PATH, safeurl(drive[i].name + '_' + drive[i].id) + '.docx'), Buffer.from(data))
      } catch (e) {
        console.log(e)
      }
    }

    if(drive[i].mimeType == 'application/vnd.google-apps.spreadsheet') {
      console.log('Exporting ' + drive[i].name)
      try {
        let response = await convertGoogleDoc(drive[i].id, 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
        let data = await response.data.arrayBuffer()
        fs.writeFileSync(path.join(DOWNLOAD_PATH, safeurl(drive[i].name + '_' + drive[i].id) + '.xlsx'), Buffer.from(data))
      } catch (e) {
        console.log(e)
      }
    }
    // TODO: application/vnd.openxmlformats-officedocument.presentationml.presentation
  }
}

module.exports = {
  downloadDocs,
  convertGoogleDoc,
  
}



how to sign a file url for uploading to google storage?


In [ ]:
const {Storage} = require('@google-cloud/storage');
const storage = new Storage();

/**
 * HTTP function that generates a signed URL
 * The signed URL can be used to upload files to Google Cloud Storage (GCS)
 *
 * @param {Object} req Cloud Function request context.
 * @param {Object} res Cloud Function response context.
 */
exports.getSignedUrl = (req, res) => {
  if (req.method !== 'POST') {
    // Return a "method not allowed" error
    return res.status(405).end();
  }
  // TODO(developer) check that the user is authorized to upload

  // Get a reference to the destination file in GCS
  const file = storage.bucket(req.body.bucket).file(req.body.filename);

  // Create a temporary upload URL
  const expiresAtMs = Date.now() + 300000; // Link expires in 5 minutes
  const config = {
    action: 'write',
    expires: expiresAtMs,
    contentType: req.body.contentType,
  };

  file.getSignedUrl(config, (err, url) => {
    if (err) {
      console.error(err);
      res.status(500).end();
      return;
    }
    res.send(url);
  });
};




### code marketing sheet



create a copy of study sauce template?


In [ ]:
var uuid = require('uuid/v1');
var importer = require('../Core');
var getSheet = importer.import('get sheet identifier');
var copyFile = importer.import('copy a file on google drive');
var listDrive = importer.import('list google drive files');
var insertPermission = importer.import('insert google drive permissions');

function copyStudy(email) {
    var fileId, newId;
    if(!email) {
        throw new Error('email not specified!');
    }
    return listDrive()
        .then(r => fileId = r.filter(f => f.name === 'Study sauce template')[0].id)
        .then(() => copyFile(fileId, 'Study sauce ' + uuid().substr(0, 5)))
        .then(id => newId = id)
        .then(() => insertPermission(newId, email))
        .then(() => getSheet(newId, null, email))
        .then(() => newId)
}

module.exports = copyStudy;



#### the code

create a sheet handler?

create a copy of marketing template?


In [ ]:
var uuid = require('uuid/v1');
var importer = require('../Core');
var getSheet = importer.import('get sheet identifier');
var copyFile = importer.import('copy a file on google drive');
var listDrive = importer.import('list google drive files');
var insertPermission = importer.import('insert google drive permissions');

function copyMarketing(email) {
    var fileId;
    return listDrive()
        .then(r => fileId = r.filter(f => f.name === 'Marketing site')[0].id)
        .then(() => copyFile(fileId, 'Marketing site ' + uuid().substr(0, 5)))
        .then(id => insertPermission(id, email))
        .then(() => getSheet(fileId, email))
        .then(() => fileId)
}

module.exports = copyMarketing;

